In [ ]:
import numpy as np

def dB_to_lin(pow_dB):
    return 10**(np.array(pow_dB)/10)

def lin_to_dB(pow_lin):
    return 10*np.log10(np.array(pow_lin))

rssi_3470_3510 = np.array([-85.53,
-95.92,
-96.29,
-101.19,
-93.44,
-99.88,
-88.62,
-99.78,
-96.45,
-88.8
])


rssi_3610_3650 = np.array([-84.93,
-98.39,
-101.84,
-101.58,
-92.01,
-101.67,
-86.77,
-98.45,
-102.38,
-89.61
])

rssi_2160_2170 = np.array([-85.43,
-76.88,
-76.22,
-86.68,
-78.85,
-80.78,
-79.53,
-74.71,
-80.88,
-72.33
])



In [ ]:
import os
import scipy.io as sio
import matplotlib.pyplot as plt
map_folderdir = "./"
directory = os.listdir(map_folderdir)
flag = 0
for fname in directory:
    if "SLCmap" in fname:
        map_file = os.path.join(map_folderdir, fname)
        flag = 1

if flag == 0:
    errorMessage = 'Error: The file does not exist in the folder:\n ' + map_folderdir
    warnings.warn(errorMessage)

print('Now reading ' + map_file + "\n")
x = sio.loadmat(map_file)
map_struct = x['SLC']

# Define a new struct named SLC
SLC = map_struct[0][0]
column_map = dict(zip([name for name in SLC.dtype.names], [i for i in range(len(SLC.dtype.names))]))

map_ = SLC[column_map["dem"]] + 0.3048 * SLC[column_map["hybrid_bldg"]]
map_ = map_[::10, ::10]

### Implement Pseudo-Inverse

In [ ]:
data_points = np.array([
    [966, 2992],
    [2569, 3767],
    [2873, 3447],
    [2621, 4286],
    [1312, 3830],
    [3828, 2667],
    [242, 2442],
    [1711, 3145],
    [2852, 1584],
    [1903, 2393]
])
data_points = data_points.astype(float)/10.0
data_points = np.floor(data_points).astype("int")

In [ ]:
def serialize_index(row, col, num_cols=map_.shape[1]):
    return row * num_cols + col

dp_serial = []
for dp in data_points:
    serial_dp = serialize_index(dp[1], dp[0])
    dp_serial.append(serial_dp)
    
dp_serial = np.array(dp_serial)
dp_serial

In [ ]:
from tqdm import tqdm
def deserialize_index(serialized_index, num_cols=map_.shape[1]):
    row = serialized_index // num_cols
    col = serialized_index % num_cols
    return row, col

def euclidean_distance(serialized_index1, serialized_index2, num_cols=map_.shape[1]):
    row1, col1 = deserialize_index(serialized_index1, num_cols)
    row2, col2 = deserialize_index(serialized_index2, num_cols)

    distance = ((row2 - row1)**2 + (col2 - col1)**2)**0.5*(0.5*10)
    if distance<1:
        distance=1
    return distance

In [ ]:
import numpy as np
from scipy.optimize import minimize

def squared_error(T_i, P_i0, n_p, d_ij, P_j):
    """
    Calculate the squared error for a given Tx power T_i.

    :param T_i: Unknown Tx power.
    :param P_i0: Reference power at a 1-meter distance from the Tx.
    :param n_p: Path loss exponent.
    :param d_ij: Array of distances between Tx i and Rx j.
    :param P_j: Array of measured powers at Rx j.
    :return: Squared error.
    """
    # Calculate modeled received power for each sensor
    P_hat_j = T_i - P_i0 - 10 * n_p * np.log10(d_ij)

    # Calculate squared error
    sqr_error = np.sum((P_hat_j - P_j)**2)
    return sqr_error

def find_minimizer_Ti(P_j, n_p, d_ij, P_i0):
    """
    Find the minimizer T_i that minimizes the squared error.

    :param P_j: Array of measured powers at Rx j.
    :param n_p: Path loss exponent.
    :param d_ij: Array of distances between Tx i and Rx j.
    :param P_i0: Reference power at a 1-meter distance from the Tx.
    :return: The minimizer T_i.
    """
    # Initial guess for T_i
    initial_T_i = 0

    # Minimize the squared error with respect to T_i
    result = minimize(squared_error, initial_T_i, args=(P_i0, n_p, d_ij, P_j))

    # Return the minimizer T_i
    return result.x[0]






In [ ]:
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed

P_j = rssi_3470_3510
n_p = 2  # Example path loss exponent
P_i0 = 0  # Example reference power

# Assuming the existence of the necessary variables like P_j, n_p, d_ij, P_i0, and data_points

def serialize_index(row_col, num_cols=map_.shape[1]):
    return row_col[:, 1] * num_cols + row_col[:, 0]

dp_serial = serialize_index(np.array(data_points))

def deserialize_index(serialized_indices, num_cols=map_.shape[1]):
    rows = serialized_indices // num_cols
    cols = serialized_indices % num_cols
    return np.vstack((rows, cols)).T

dp_coords = deserialize_index(dp_serial)

def euclidean_distance(point, dp_coords):
    distances = np.linalg.norm(dp_coords - point, axis=1) * 0.5*10
    distances[distances < 0.5*10] = 0.5*10
    return distances

def process_batch(batch_indices, dp_coords, map_shape, num_cols):
    minimizer_T_batch = []
    for i in batch_indices:
        point = deserialize_index(np.array([i]), num_cols)[0]
        distances = euclidean_distance(point, dp_coords)
        minimizer_T_batch.append(find_minimizer_Ti(P_j, n_p, distances, P_i0))
    return minimizer_T_batch

# Parallel processing
num_cores = -1  # Adjust as needed. -1 uses all available cores
batch_size = 100  # Adjust based on memory and performance considerations

map_indices = np.arange(map_.size)
batches = [map_indices[i:i + batch_size] for i in range(0, len(map_indices), batch_size)]

minimizer_T_batches = Parallel(n_jobs=num_cores)(
    delayed(process_batch)(batch, dp_coords, map_.shape, map_.shape[1]) for batch in tqdm(batches))

minimizer_T = [item for sublist in minimizer_T_batches for item in sublist]
minimizer_Ts = np.reshape(minimizer_T, map_.shape)

np.save("minimizer_T_3470_3510_low_res3.npy",minimizer_Ts)

In [ ]:
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed

P_j = rssi_3610_3650
n_p = 2  # Example path loss exponent
P_i0 = 0  # Example reference power

# Assuming the existence of the necessary variables like P_j, n_p, d_ij, P_i0, and data_points

def serialize_index(row_col, num_cols=map_.shape[1]):
    return row_col[:, 1] * num_cols + row_col[:, 0]

dp_serial = serialize_index(np.array(data_points))

def deserialize_index(serialized_indices, num_cols=map_.shape[1]):
    rows = serialized_indices // num_cols
    cols = serialized_indices % num_cols
    return np.vstack((rows, cols)).T

dp_coords = deserialize_index(dp_serial)

def euclidean_distance(point, dp_coords):
    distances = np.linalg.norm(dp_coords - point, axis=1) * 0.5*10
    distances[distances < 0.5*10] = 0.5*10
    return distances

def process_batch(batch_indices, dp_coords, map_shape, num_cols):
    minimizer_T_batch = []
    for i in batch_indices:
        point = deserialize_index(np.array([i]), num_cols)[0]
        distances = euclidean_distance(point, dp_coords)
        minimizer_T_batch.append(find_minimizer_Ti(P_j, n_p, distances, P_i0))
    return minimizer_T_batch

# Parallel processing
num_cores = -1  # Adjust as needed. -1 uses all available cores
batch_size = 100  # Adjust based on memory and performance considerations

map_indices = np.arange(map_.size)
batches = [map_indices[i:i + batch_size] for i in range(0, len(map_indices), batch_size)]

minimizer_T_batches = Parallel(n_jobs=num_cores)(
    delayed(process_batch)(batch, dp_coords, map_.shape, map_.shape[1]) for batch in tqdm(batches))

minimizer_T = [item for sublist in minimizer_T_batches for item in sublist]
minimizer_Ts = np.reshape(minimizer_T, map_.shape)

np.save("minimizer_T_3610_3650_low_res3.npy",minimizer_Ts)

In [ ]:
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed

P_j = rssi_2504_2544
n_p = 3  # Example path loss exponent
P_i0 = 0  # Example reference power

# Assuming the existence of the necessary variables like P_j, n_p, d_ij, P_i0, and data_points

def serialize_index(row_col, num_cols=map_.shape[1]):
    return row_col[:, 1] * num_cols + row_col[:, 0]

dp_serial = serialize_index(np.array(data_points))

def deserialize_index(serialized_indices, num_cols=map_.shape[1]):
    rows = serialized_indices // num_cols
    cols = serialized_indices % num_cols
    return np.vstack((rows, cols)).T

dp_coords = deserialize_index(dp_serial)

def euclidean_distance(point, dp_coords):
    distances = np.linalg.norm(dp_coords - point, axis=1) * 0.5*10
    distances[distances < 0.5*10] = 0.5*10
    return distances

def process_batch(batch_indices, dp_coords, map_shape, num_cols):
    minimizer_T_batch = []
    for i in batch_indices:
        point = deserialize_index(np.array([i]), num_cols)[0]
        distances = euclidean_distance(point, dp_coords)
        minimizer_T_batch.append(find_minimizer_Ti(P_j, n_p, distances, P_i0))
    return minimizer_T_batch

# Parallel processing
num_cores = -1  # Adjust as needed. -1 uses all available cores
batch_size = 100  # Adjust based on memory and performance considerations

map_indices = np.arange(map_.size)
batches = [map_indices[i:i + batch_size] for i in range(0, len(map_indices), batch_size)]

minimizer_T_batches = Parallel(n_jobs=num_cores)(
    delayed(process_batch)(batch, dp_coords, map_.shape, map_.shape[1]) for batch in tqdm(batches))

minimizer_T = [item for sublist in minimizer_T_batches for item in sublist]
minimizer_Ts = np.reshape(minimizer_T, map_.shape)

np.save("minimizer_T_2504_2544_low_res.npy",minimizer_Ts)

In [ ]:
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed

P_j = rssi_5190_5210
n_p = 3  # Example path loss exponent
P_i0 = 0  # Example reference power

# Assuming the existence of the necessary variables like P_j, n_p, d_ij, P_i0, and data_points

def serialize_index(row_col, num_cols=map_.shape[1]):
    return row_col[:, 1] * num_cols + row_col[:, 0]

dp_serial = serialize_index(np.array(data_points))

def deserialize_index(serialized_indices, num_cols=map_.shape[1]):
    rows = serialized_indices // num_cols
    cols = serialized_indices % num_cols
    return np.vstack((rows, cols)).T

dp_coords = deserialize_index(dp_serial)

def euclidean_distance(point, dp_coords):
    distances = np.linalg.norm(dp_coords - point, axis=1) * 0.5*10
    distances[distances < 0.5*10] = 0.5*10
    return distances

def process_batch(batch_indices, dp_coords, map_shape, num_cols):
    minimizer_T_batch = []
    for i in batch_indices:
        point = deserialize_index(np.array([i]), num_cols)[0]
        distances = euclidean_distance(point, dp_coords)
        minimizer_T_batch.append(find_minimizer_Ti(P_j, n_p, distances, P_i0))
    return minimizer_T_batch

# Parallel processing
num_cores = -1  # Adjust as needed. -1 uses all available cores
batch_size = 100  # Adjust based on memory and performance considerations

map_indices = np.arange(map_.size)
batches = [map_indices[i:i + batch_size] for i in range(0, len(map_indices), batch_size)]

minimizer_T_batches = Parallel(n_jobs=num_cores)(
    delayed(process_batch)(batch, dp_coords, map_.shape, map_.shape[1]) for batch in tqdm(batches))

minimizer_T = [item for sublist in minimizer_T_batches for item in sublist]
minimizer_Ts = np.reshape(minimizer_T, map_.shape)

np.save("minimizer_T_5190_5210_low_res.npy",minimizer_Ts)

In [ ]:
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed

P_j = rssi_2160_2170
n_p = 2  # Example path loss exponent
P_i0 = 0  # Example reference power

# Assuming the existence of the necessary variables like P_j, n_p, d_ij, P_i0, and data_points

def serialize_index(row_col, num_cols=map_.shape[1]):
    return row_col[:, 1] * num_cols + row_col[:, 0]

dp_serial = serialize_index(np.array(data_points))

def deserialize_index(serialized_indices, num_cols=map_.shape[1]):
    rows = serialized_indices // num_cols
    cols = serialized_indices % num_cols
    return np.vstack((rows, cols)).T

dp_coords = deserialize_index(dp_serial)

def euclidean_distance(point, dp_coords):
    distances = np.linalg.norm(dp_coords - point, axis=1) * 0.5*10
    distances[distances < 0.5*10] = 0.5*10
    return distances

def process_batch(batch_indices, dp_coords, map_shape, num_cols):
    minimizer_T_batch = []
    for i in batch_indices:
        point = deserialize_index(np.array([i]), num_cols)[0]
        distances = euclidean_distance(point, dp_coords)
        minimizer_T_batch.append(find_minimizer_Ti(P_j, n_p, distances, P_i0))
    return minimizer_T_batch

# Parallel processing
num_cores = -1  # Adjust as needed. -1 uses all available cores
batch_size = 100  # Adjust based on memory and performance considerations

map_indices = np.arange(map_.size)
batches = [map_indices[i:i + batch_size] for i in range(0, len(map_indices), batch_size)]

minimizer_T_batches = Parallel(n_jobs=num_cores)(
    delayed(process_batch)(batch, dp_coords, map_.shape, map_.shape[1]) for batch in tqdm(batches))

minimizer_T = [item for sublist in minimizer_T_batches for item in sublist]
minimizer_Ts = np.reshape(minimizer_T, map_.shape)

np.save("minimizer_T_2160_2170_low_res2.npy",minimizer_Ts)

In [ ]:
distances

In [ ]:
minimizer_Ts = np.load("minimizer_T_2160_2170_low_res2.npy")
x = np.linspace(0, map_.shape[1], map_.shape[1], endpoint=False)
y = np.linspace(0, map_.shape[0], map_.shape[0], endpoint=False)
X, Y = np.meshgrid(x, y)


plt.figure(figsize=(13, 8))
plt.contourf(X, Y, minimizer_Ts, 100, cmap='viridis')  # Use the masked array here
plt.colorbar(label='dBX')

# Plot the data collection points for reference
scatter = plt.scatter(data_points[:, 0], data_points[:, 1], c=rssi_2160_2170, s=50, cmap='plasma', label='Data Collection Points')
cbar = plt.colorbar(scatter, label='Signal Strength (dBX)', location='left')  # Colorbar for signal strengths on the left
plt.legend()

plt.xlabel('UTM_E [m]')
plt.ylabel('UTM_N [m]')
plt.title('2D Predictions of Signal Strength')
plt.show()


In [ ]:
minimizer_Ts = np.load("minimizer_T_3610_3650_low_res2.npy")
x = np.linspace(0, map_.shape[1], map_.shape[1], endpoint=False)
y = np.linspace(0, map_.shape[0], map_.shape[0], endpoint=False)
X, Y = np.meshgrid(x, y)


plt.figure(figsize=(13, 8))
plt.contourf(X, Y, minimizer_Ts, 100, cmap='viridis')  # Use the masked array here
plt.colorbar(label='dBX')

# Plot the data collection points for reference
scatter = plt.scatter(data_points[:, 0], data_points[:, 1], c=rssi_3610_3650, s=50, cmap='plasma', label='Data Collection Points')
cbar = plt.colorbar(scatter, label='Signal Strength (dBX)', location='left')  # Colorbar for signal strengths on the left
plt.legend()

plt.xlabel('UTM_E [m]')
plt.ylabel('UTM_N [m]')
plt.title('2D Predictions of Signal Strength')
plt.show()


In [ ]:
minimizer_Ts = np.load("minimizer_T_2504_2544_low_res.npy")

x = np.linspace(0, map_.shape[1], map_.shape[1], endpoint=False)
y = np.linspace(0, map_.shape[0], map_.shape[0], endpoint=False)
X, Y = np.meshgrid(x, y)


plt.figure(figsize=(13, 8))
plt.contourf(X, Y, minimizer_Ts, 100, cmap='viridis')  # Use the masked array here
plt.colorbar(label='dBX')

# Plot the data collection points for reference
scatter = plt.scatter(data_points[:, 0], data_points[:, 1], c=rssi_2504_2544, s=50, cmap='plasma', label='Data Collection Points')
cbar = plt.colorbar(scatter, label='Signal Strength (dBX)', location='left')  # Colorbar for signal strengths on the left
plt.legend()

plt.xlabel('UTM_E [m]')
plt.ylabel('UTM_N [m]')
plt.title('2D Predictions of Signal Strength')
plt.show()


In [ ]:
minimizer_Ts = np.load("minimizer_T_5190_5210_low_res.npy")

x = np.linspace(0, map_.shape[1], map_.shape[1], endpoint=False)
y = np.linspace(0, map_.shape[0], map_.shape[0], endpoint=False)
X, Y = np.meshgrid(x, y)


plt.figure(figsize=(13, 8))
plt.contourf(X, Y, minimizer_Ts, 100, cmap='viridis')  # Use the masked array here
plt.colorbar(label='dBX')

# Plot the data collection points for reference
scatter = plt.scatter(data_points[:, 0], data_points[:, 1], c=rssi_5190_5210, s=50, cmap='plasma', label='Data Collection Points')
cbar = plt.colorbar(scatter, label='Signal Strength (dBX)', location='left')  # Colorbar for signal strengths on the left
plt.legend()

plt.xlabel('UTM_E [m]')
plt.ylabel('UTM_N [m]')
plt.title('2D Predictions of Signal Strength')
plt.show()


In [ ]:
minimizer_Ts = np.load("minimizer_T_3470_3510_low_res.npy")

def deserialize_index(serialized_index, num_cols=map_.shape[1]):
    row = serialized_index // num_cols
    col = serialized_index % num_cols
    return row, col

def euclidean_distance(serialized_index1, serialized_index2, num_cols=map_.shape[1]):
    row1, col1 = deserialize_index(serialized_index1, num_cols)
    row2, col2 = deserialize_index(serialized_index2, num_cols)

    distance = ((row2 - row1)**2 + (col2 - col1)**2)**0.5*(0.5*10)
    if distance<0.5*10:
        distance=0.5*10
    return distance

signal_strengths = rssi_3470_3510

sigma = 2.5
sigma_x = 5
d_c = 50
C = np.zeros((len(data_points),len(data_points)))
for i in range(len(data_points)):
    for j in range(len(data_points)):
        C[i, j] = sigma_x**2*np.exp(-euclidean_distance(dp_serial[i], dp_serial[j])/d_c)

C_inv = np.linalg.inv(C)
min_T = minimizer_Ts.ravel()
map_ = np.zeros((map_.shape[0], map_.shape[1]))
#mean_sqr_err = np.zeros((len(map_.ravel()),))

f_xz_new = np.zeros((len(map_.ravel()),))
for i in tqdm(range(len(map_.ravel()))):
    d_ij = []
    for j in range(len(dp_serial)):
        d_ij.append(euclidean_distance(i, dp_serial[j]))
    diff = min_T[i]-10*n_p*np.log10(d_ij)-signal_strengths
    f_xz_new[i] = np.exp(-1/2*diff.T@C_inv@diff)
    #mean_sqr_err[i] = np.mean(diff**2)


        
#f_xz = np.exp(-mean_sqr_err/(2*sigma**2))
f_xz_new /= np.sum(f_xz_new)

f_xz_2d = np.reshape(f_xz_new, map_.shape)

x = np.linspace(0, map_.shape[1], map_.shape[1], endpoint=False)
y = np.linspace(0, map_.shape[0], map_.shape[0], endpoint=False)
X, Y = np.meshgrid(x, y)


plt.figure(figsize=(13, 8))
plt.contourf(X, Y, f_xz_2d, 100, cmap='viridis')  # Use the masked array here
plt.colorbar(label='dBX')

# Plot the data collection points for reference
scatter = plt.scatter(data_points[:, 0], data_points[:, 1], c=signal_strengths, s=50, cmap='plasma', label='Data Collection Points')
cbar = plt.colorbar(scatter, label='Signal Strength (dBX)', location='left')  # Colorbar for signal strengths on the left
plt.legend()

plt.xlabel('UTM_E [m]')
plt.ylabel('UTM_N [m]')
plt.title('2D Predictions of Signal Strength')
plt.show()

np.save("f_xz_2d_3470_3510_low_res_cov.npy",f_xz_2d)

In [ ]:
f_xz_new

In [ ]:
x = np.linspace(0, map_.shape[1], map_.shape[1], endpoint=False)
y = np.linspace(0, map_.shape[0], map_.shape[0], endpoint=False)
X, Y = np.meshgrid(x, y)

f_xz_2d = np.load("f_xz_2d_3470_3510_low_res.npy")
plt.figure(figsize=(13, 8))
plt.contourf(X, Y, f_xz_2d, 100, cmap='viridis')  # Use the masked array here
plt.colorbar(label='Probability')

# Plot the data collection points for reference
scatter = plt.scatter(data_points[:, 0], data_points[:, 1], c=rssi_3470_3510, s=50, cmap='plasma', label='Data Collection Points')
cbar = plt.colorbar(scatter, label='Signal Strength (dBX)', location='left')  # Colorbar for signal strengths on the left
plt.legend()

plt.xlabel('UTM_E [m]')
plt.ylabel('UTM_N [m]')
plt.title('2D Probability Distribution of Transmitter Location')
plt.show()

### Calculate the Signal Strength Estimates

In [ ]:
signal_strengths = rssi_3470_3510
minimizer_Ts = np.load("minimizer_T_3470_3510_low_res.npy")
min_T = minimizer_Ts.ravel()
n_p=3
probabilities = np.load("f_xz_2d_3470_3510_low_res.npy")
probabilities = probabilities.ravel()

idx_useful = np.where(probabilities>0.3e-5)[0]
print(len(idx_useful))
print(np.sum(probabilities[idx_useful]))
signal_estimates = np.zeros_like(map_)
signal_estimates = signal_estimates.ravel()
for i in tqdm(range(len(map_.ravel()))):
    d_ij = []
    for j in idx_useful:
        d_ij.append(euclidean_distance(i, j))
    P_hats = min_T[idx_useful] - 10 * n_p * np.log10(d_ij)
    signal_estimates[i] = np.sum(probabilities[idx_useful]*dB_to_lin(P_hats))/np.sum(probabilities[idx_useful])
    if i%3000000 == 0 and i!=0:
        plt.figure(figsize=(13, 8))
        plt.contourf(X, Y, np.reshape(lin_to_dB(signal_estimates),map_.shape), 100, cmap='viridis')  # Use the masked array here
        plt.colorbar(label='dBX')

        # Plot the data collection points for reference
        scatter = plt.scatter(data_points[:, 0], data_points[:, 1], c=signal_strengths, s=50, cmap='plasma', label='Data Collection Points')
        cbar = plt.colorbar(scatter, label='Signal Strength (dBX)', location='left')  # Colorbar for signal strengths on the left
        plt.legend()

        plt.xlabel('UTM_E [m]')
        plt.ylabel('UTM_N [m]')
        plt.title('2D Predictions of Signal Strength')
        plt.show()
        


In [ ]:
import numpy as np
from scipy.spatial.distance import cdist
from tqdm import tqdm
from joblib import Parallel, delayed
import matplotlib.pyplot as plt

# Load your data
signal_strengths = rssi_3470_3510
minimizer_Ts = np.load("minimizer_T_3470_3510.npy").ravel()
probabilities = np.load("f_xz_2d_3470_3510.npy").ravel()
n_p = 3

# Identify useful indices
idx_useful = np.where(probabilities > 0.3e-6)[0]


# Dimensions of the map

# Generate meshgrid for map coordinates
Y, X = np.meshgrid(np.arange(map_.shape[1]), np.arange(map_.shape[0]))

# Flatten the meshgrid and stack them as coordinates
all_coords = np.stack((X.ravel(), Y.ravel()), axis=1)
# Convert idx_useful to 2D coordinates
useful_2d_indices = np.unravel_index(idx_useful, (map_.shape[0], map_.shape[1]))

# Stack these coordinates
useful_coords = np.stack(useful_2d_indices, axis=1)

# Now, all_coords is ready to be used in your distance calculations

# Vectorize distance calculation
distances = cdist(all_coords, useful_coords)
print()
# Function to process a chunk of data
def process_chunk(start_idx, end_idx):
    partial_signal_estimates = np.zeros(end_idx - start_idx)
    for i in range(start_idx, end_idx):
        d_ij = distances[i]
        P_hats = minimizer_Ts[idx_useful] - 10 * n_p * np.log10(d_ij)
        weighted_sum = np.sum(probabilities[idx_useful] * dB_to_lin(P_hats))
        partial_signal_estimates[i - start_idx] = weighted_sum / np.sum(probabilities[idx_useful])
    return partial_signal_estimates

# Parallel computation
num_cores = -1  # Adjust based on your CPU
chunk_size = len(all_coords) // num_cores
ranges = [(i, min(i + chunk_size, len(all_coords))) for i in range(0, len(all_coords), chunk_size)]

results = Parallel(n_jobs=num_cores)(delayed(process_chunk)(*r) for r in ranges)
signal_estimates = np.concatenate(results)

# Reshape signal_estimates if necessary and use it for plotting


In [ ]:
minimizer_Ts = np.load("minimizer_T_3470_3510_low_res.npy")
min_T = minimizer_Ts.ravel()

In [ ]:
min_T

In [ ]:
import numpy as np
from scipy.spatial.distance import cdist
from joblib import Parallel, delayed

# Assuming dB_to_lin and other necessary functions are defined
signal_strengths = rssi_5190_5210
minimizer_Ts = np.load("minimizer_T_3470_3510_low_res.npy")
min_T = minimizer_Ts.ravel()
n_p=3
probabilities = np.load("f_xz_2d_3470_3510_low_res_cov.npy")
probabilities = probabilities.ravel()
idx_useful = np.where(probabilities > 0)[0]

useful_2d_indices = np.unravel_index(idx_useful, (map_.shape[0], map_.shape[1]))
print(useful_2d_indices)
# Stack these coordinates
useful_coords = np.stack(useful_2d_indices, axis=1)
print(useful_coords[:10, :])

all_coords = useful_coords.copy()
print(all_coords[:10, :])

# Batch processing function
def process_batch(start_idx, end_idx):
    # Compute distances for a batch
    batch_distances = cdist(all_coords[start_idx:end_idx], useful_coords)*5
    # Initialize an array to hold signal estimates for this batch
    batch_signal_estimates = np.zeros(end_idx - start_idx)

    # Process distances and compute signal estimates
    for i, d_ij in enumerate(batch_distances):
        P_hats = min_T[idx_useful] - 10 * n_p * np.log10(1+d_ij)
        weighted_sum = np.sum(probabilities[idx_useful] * dB_to_lin(P_hats))
        batch_signal_estimates[i] = weighted_sum / np.sum(probabilities[idx_useful])

    return batch_signal_estimates

# Parameters for batch processing
num_cores = -1  # Adjust based on your CPU
num_points = all_coords.shape[0]
batch_size = 10  # Adjust based on your memory capacity

# Create ranges for batches
ranges = [(i, min(i + batch_size, num_points)) for i in range(0, num_points, batch_size)]
# Perform parallel batch processing
results = Parallel(n_jobs=num_cores)(delayed(process_batch)(*r) for r in ranges)

# Combine results from all batches
signal_estimates = np.concatenate(results)
np.save("signal_estimates_3470_3510_low_res_cov.npy", signal_estimates)

In [ ]:
signal_estimates = np.reshape(signal_estimates, map_.shape)
x = np.linspace(0, map_.shape[1], map_.shape[1], endpoint=False)
y = np.linspace(0, map_.shape[0], map_.shape[0], endpoint=False)
X, Y = np.meshgrid(x, y)


plt.figure(figsize=(13, 8))
plt.contourf(X, Y, lin_to_dB(signal_estimates), 100, cmap='viridis')  # Use the masked array here
plt.colorbar(label='dBX')

# Plot the data collection points for reference
scatter = plt.scatter(data_points[:, 0], data_points[:, 1], c=signal_strengths, s=50, cmap='plasma', label='Data Collection Points')
cbar = plt.colorbar(scatter, label='Signal Strength (dBX)', location='left')  # Colorbar for signal strengths on the left
plt.legend()

plt.xlabel('UTM_E [m]')
plt.ylabel('UTM_N [m]')
plt.title('2D Predictions of Signal Strength')
plt.show()

In [ ]:
signal_estimates = np.reshape(signal_estimates, map_.shape)
x = np.linspace(0, map_.shape[1], map_.shape[1], endpoint=False)
y = np.linspace(0, map_.shape[0], map_.shape[0], endpoint=False)
X, Y = np.meshgrid(x, y)


plt.figure(figsize=(13, 8))
plt.contourf(X, Y, lin_to_dB(signal_estimates), 100, cmap='viridis')  # Use the masked array here
plt.colorbar(label='dBX')

# Plot the data collection points for reference
scatter = plt.scatter(data_points[:, 0], data_points[:, 1], c=signal_strengths, s=50, cmap='plasma', label='Data Collection Points')
cbar = plt.colorbar(scatter, label='Signal Strength (dBX)', location='left')  # Colorbar for signal strengths on the left
plt.legend()

plt.xlabel('UTM_E [m]')
plt.ylabel('UTM_N [m]')
plt.title('2D Predictions of Signal Strength')
plt.show()

In [ ]:
signal_estimates = np.reshape(signal_estimates, map_.shape)
x = np.linspace(0, map_.shape[1], map_.shape[1], endpoint=False)
y = np.linspace(0, map_.shape[0], map_.shape[0], endpoint=False)
X, Y = np.meshgrid(x, y)


plt.figure(figsize=(13, 8))
plt.contourf(X, Y, lin_to_dB(signal_estimates), 100, cmap='viridis')  # Use the masked array here
plt.colorbar(label='Predicted Signal Strength (dBX)')

# Plot the data collection points for reference
scatter = plt.scatter(data_points[:, 0], data_points[:, 1], c=signal_strengths, s=50, cmap='plasma', label='Data Collection Points')
cbar = plt.colorbar(scatter, label='Observed Signal Strength (dBX)', location='left')  # Colorbar for signal strengths on the left
plt.legend()

plt.xlabel('UTM_E [m]')
plt.ylabel('UTM_N [m]')
plt.title('2D Predictions of Signal Strength')
plt.show()

In [ ]:
signal_estimates = np.reshape(signal_estimates, map_.shape)
x = np.linspace(0, map_.shape[1], map_.shape[1], endpoint=False)
y = np.linspace(0, map_.shape[0], map_.shape[0], endpoint=False)
X, Y = np.meshgrid(x, y)


plt.figure(figsize=(13, 8))
plt.contourf(X, Y, lin_to_dB(signal_estimates), 100, cmap='viridis')  # Use the masked array here
plt.colorbar(label='dBX')

# Plot the data collection points for reference
scatter = plt.scatter(data_points[:, 0], data_points[:, 1], c=signal_strengths, s=50, cmap='plasma', label='Data Collection Points')
cbar = plt.colorbar(scatter, label='Signal Strength (dBX)', location='left')  # Colorbar for signal strengths on the left
plt.legend()

plt.xlabel('UTM_E [m]')
plt.ylabel('UTM_N [m]')
plt.title('2D Predictions of Signal Strength')
plt.show()

In [ ]:
has_nan_or_inf = np.any(np.isinf(signal_estimates))

In [ ]:
has_nan_or_inf

In [ ]:
from sklearn.metrics import mean_squared_error
scores = []
signal_strengths = rssi_3470_3510
for i in range(len(data_points)):
    train_points = np.delete(data_points, i, axis=0)
    train_strengths = np.delete(signal_strengths, i, axis=0)
    test_point = data_points[i, :].reshape(1, -1)
    test_strength = signal_strengths[i]
    predicted_strength = lin_to_dB(signal_estimates[data_points[i][1], data_points[i][0]])
    mse = (test_strength-predicted_strength)**2
    scores.append(mse)
print(np.mean(scores))
print(np.std(scores))

In [ ]:
np.std(signal_strengths)**2

In [ ]:
import numpy as np
from scipy.spatial.distance import cdist
from joblib import Parallel, delayed
from sklearn.metrics import mean_squared_error
def deserialize_index(serialized_index, num_cols=map_.shape[1]):
    row = serialized_index // num_cols
    col = serialized_index % num_cols
    return row, col

def euclidean_distance(serialized_index1, serialized_index2, num_cols=map_.shape[1]):
    row1, col1 = deserialize_index(serialized_index1, num_cols)
    row2, col2 = deserialize_index(serialized_index2, num_cols)

    distance = ((row2 - row1)**2 + (col2 - col1)**2)**0.5*(0.5*10)
    if distance<0.5*10:
        distance=0.5*10
    return distance

import os
import scipy.io as sio
import matplotlib.pyplot as plt
map_folderdir = "./"
directory = os.listdir(map_folderdir)
flag = 0
for fname in directory:
    if "SLCmap" in fname:
        map_file = os.path.join(map_folderdir, fname)
        flag = 1

if flag == 0:
    errorMessage = 'Error: The file does not exist in the folder:\n ' + map_folderdir
    warnings.warn(errorMessage)

print('Now reading ' + map_file + "\n")
x = sio.loadmat(map_file)
map_struct = x['SLC']

# Define a new struct named SLC
SLC = map_struct[0][0]
column_map = dict(zip([name for name in SLC.dtype.names], [i for i in range(len(SLC.dtype.names))]))

map_ = SLC[column_map["dem"]] + 0.3048 * SLC[column_map["hybrid_bldg"]]
map_ = map_[::10, ::10]


def optimize_parameters(band, sigma_x, d_c, n_p, map_):
    minimizer_Ts = np.load("minimizer_T_"+band+"_low_res3.npy")

    if band == "3470_3510":
        signal_strengths = rssi_3470_3510
    if band == "3610_3650":
        signal_strengths = rssi_3610_3650
    if band == "2504_2544":
        signal_strengths = rssi_2504_2544
    if band == "2160_2170":
        signal_strengths = rssi_2160_2170


    C = np.zeros((len(data_points),len(data_points)))
    for i in range(len(data_points)):
        for j in range(len(data_points)):
            C[i, j] = sigma_x**2*np.exp(-euclidean_distance(dp_serial[i], dp_serial[j])/d_c)

    C_inv = np.linalg.inv(C)
    min_T = minimizer_Ts.ravel()
    map_ = np.zeros((map_.shape[0], map_.shape[1]))
    #mean_sqr_err = np.zeros((len(map_.ravel()),))

    f_xz_new = np.zeros((len(map_.ravel()),))
    for i in range(len(map_.ravel())):
        d_ij = []
        for j in range(len(dp_serial)):
            d_ij.append(euclidean_distance(i, dp_serial[j]))
        diff = min_T[i]-10*n_p*np.log10(d_ij)-signal_strengths
        f_xz_new[i] = np.exp(-1/2*diff.T@C_inv@diff)
        #mean_sqr_err[i] = np.mean(diff**2)



    #f_xz = np.exp(-mean_sqr_err/(2*sigma**2))
    f_xz_new /= np.sum(f_xz_new)
    if True in np.isnan(f_xz_new):
        print("Results' length is zero, skipping.")
        print(f"Band: {band}, Sigma_x: {sigma_x}, d_c: {d_c}, n_p: {n_p}")
        return None
    f_xz_2d = np.reshape(f_xz_new, map_.shape)
    np.save("f_xz_2d_"+band+"_low_res_cov3.npy",f_xz_2d)
    
    x = np.linspace(0, map_.shape[1], map_.shape[1], endpoint=False)
    y = np.linspace(0, map_.shape[0], map_.shape[0], endpoint=False)
    X, Y = np.meshgrid(x, y)


    plt.figure(figsize=(13, 8))
    plt.contourf(X, Y, f_xz_2d, 100, cmap='viridis')  # Use the masked array here
    plt.colorbar(label='dBX')

    # Plot the data collection points for reference
    scatter = plt.scatter(data_points[:, 0], data_points[:, 1], c=signal_strengths, s=50, cmap='plasma', label='Data Collection Points')
    cbar = plt.colorbar(scatter, label='Signal Strength (dBX)', location='left')  # Colorbar for signal strengths on the left
    plt.legend()

    plt.xlabel('UTM_E [m]')
    plt.ylabel('UTM_N [m]')
    plt.title('2D Predictions of Signal Strength')
    plt.show()


    # Assuming dB_to_lin and other necessary functions are defined
    #probabilities = np.load("f_xz_2d_3470_3510_low_res_cov.npy")
    probabilities=f_xz_2d
    probabilities = probabilities.ravel()
    idx_useful = np.where(probabilities > 0)[0]

    useful_2d_indices = np.unravel_index(idx_useful, (map_.shape[0], map_.shape[1]))
    # Stack these coordinates
    useful_coords = np.stack(useful_2d_indices, axis=1)

    all_coords = useful_coords.copy()

    # Batch processing function
    def process_batch(start_idx, end_idx):
        # Compute distances for a batch
        batch_distances = cdist(all_coords[start_idx:end_idx], useful_coords)*5
        # Initialize an array to hold signal estimates for this batch
        batch_signal_estimates = np.zeros(end_idx - start_idx)

        # Process distances and compute signal estimates
        for i, d_ij in enumerate(batch_distances):
            P_hats = min_T[idx_useful] - 10 * n_p * np.log10(1+d_ij)
            weighted_sum = np.sum(probabilities[idx_useful] * dB_to_lin(P_hats))
            batch_signal_estimates[i] = weighted_sum / np.sum(probabilities[idx_useful])

        return batch_signal_estimates

    # Parameters for batch processing
    num_cores = -1  # Adjust based on your CPU
    num_points = all_coords.shape[0]
    batch_size = 10  # Adjust based on your memory capacity

    # Create ranges for batches
    ranges = [(i, min(i + batch_size, num_points)) for i in range(0, num_points, batch_size)]
    # Perform parallel batch processing
    results = Parallel(n_jobs=num_cores)(delayed(process_batch)(*r) for r in ranges)

    # Combine results from all batches
    signal_estimates = np.concatenate(results)
    np.save("signal_estimates_"+band+"_low_res_cov3.npy", signal_estimates)
    if len(signal_estimates)!=map_.shape[1]*map_.shape[0]:
        print("Results' length is zero, skipping.")
        print(f"Band: {band}, Sigma_x: {sigma_x}, d_c: {d_c}, n_p: {n_p}")
        return None
    
    signal_estimates = np.reshape(signal_estimates, map_.shape)
    
    plt.figure(figsize=(13, 8))
    plt.contourf(X, Y, lin_to_dB(signal_estimates), 100, cmap='viridis')  # Use the masked array here
    plt.colorbar(label='dBX')

    # Plot the data collection points for reference
    scatter = plt.scatter(data_points[:, 0], data_points[:, 1], c=signal_strengths, s=50, cmap='plasma', label='Data Collection Points')
    cbar = plt.colorbar(scatter, label='Signal Strength (dBX)', location='left')  # Colorbar for signal strengths on the left
    plt.legend()

    plt.xlabel('UTM_E [m]')
    plt.ylabel('UTM_N [m]')
    plt.title('2D Predictions of Signal Strength')
    plt.show()
    
    scores = []
    for i in range(len(data_points)):
        train_points = np.delete(data_points, i, axis=0)
        train_strengths = np.delete(signal_strengths, i, axis=0)
        test_point = data_points[i, :].reshape(1, -1)
        test_strength = signal_strengths[i]
        predicted_strength = lin_to_dB(signal_estimates[data_points[i][1], data_points[i][0]])
        mse = (test_strength-predicted_strength)**2
        scores.append(mse)
    print(f"Band: {band}, Sigma_x: {sigma_x}, d_c: {d_c}, n_p: {n_p}, score: {np.mean(scores)}")
    return np.mean(scores)


for band in ["3470_3510", "3610_3650"]:
    for sigma_x in [4.5]:
        for d_c in [400]:
            optimize_parameters(band, sigma_x, d_c, 2, map_)

In [ ]:
f_xz_new

In [ ]:
32653/521

In [ ]:
a = [1,2,3]
4 in a

In [ ]:
np.var(rssi_3470_3510)

In [ ]:
np.var(rssi_3610_3650)

In [ ]:
np.var(rssi_2160_2170)